In [ ]:
# 부채총계(연결), 자산총계(연결), , 자본총계(연결), 전년도자산총계(연결), 영업활동으로인한현금흐름(연결), 총당기순이익(연결), 유형자산(계)(연결), 매출채권(연결), 총수익(연결), 영업이익(손실)(연결), 매출총이익(손실)(연결)

#############################################################
######################파일 불러오기###########################
#############################################################
file = '이름.csv'
df = pd.read_csv(file, encoding = 'utf-8')


company_job = str(input())

In [2]:
############################################ 테스트 용 ########################################

import pandas as pd

# 예시 데이터프레임 생성
data = {
    '부채총계(연결)': [100],
    '자산총계(연결)': [500],
    '자본총계(연결)': [400],
    '전년도자산총계(연결)': [450],
    '영업활동으로인한현금흐름(연결)': [50],
    '총당기순이익(연결)': [30],
    '유형자산(계)(연결)': [150],
    '매출채권(연결)': [80],
    '매출액(연결)': [700],
    '영업이익(손실)(연결)': [20],
    '매출총이익(손실)(연결)': [60],
    '매출원가(손실)(연결)': [70],
    '유동부채':[70]
}

df = pd.DataFrame(data)

# '부채총계'와 '자산총계'를 포함한 컬럼을 찾기
debt_column = [col for col in df.columns if '부채총계' in col][0]
asset_column = [col for col in df.columns if '자산총계' in col][0]

# 해당 컬럼들의 시리즈를 들고와서 계산
debt_series = df[debt_column]
asset_series = df[asset_column]

# 부채총계 시리즈 / 자산총계 시리즈 계산
ratio_series = debt_series / asset_series
pd.DataFrame(ratio_series)
print(pd.DataFrame(ratio_series))

     0
0  0.2


In [29]:
#############################################################################################################################
############################################### 필요 라이브러리 불러오기 ######################################################
#############################################################################################################################

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import openai
import os

import warnings
warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

# 시각화 한글폰트 설정
plt.rc('font', family='Malgun')
sns.set(font="Malgun Gothic",
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.
import joblib
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Flatten, SimpleRNN, LSTM, GRU
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import joblib

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

#############################################################################################################################
############################################### 시계열 모델 형식 만들기 #######################################################
#############################################################################################################################

def make_dataframe(df, target_col):
    period_length = 9                            # 2015 ~ 2024
    num_sets = len(df) // period_length
    frames = []
    for i in range(num_sets):
        sub_df = df.iloc[i * period_length : (i + 1) * period_length].reset_index(drop=True)
        sub_df['target_col'] = sub_df['target_col'].shift(-1)
        sub_df = sub_df.dropna().reset_index(drop=True)
        frames.append(sub_df)
    result_df = pd.concat(frames, ignore_index=True)
    return result_df

#############################################################################################################################
########################################### 시계열 데이터 형식 만들기 #########################################################
#############################################################################################################################

def temporalize(x, timesteps):
    nfeature = x.shape[1]
    output_x = []
    for i in range(len(x) - timesteps + 1):
        t = []
        for j in range(timesteps):
            t.append(x[[(i + j)], :])
        output_x.append(t)
    return np.array(output_x).reshape(-1, timesteps, nfeature)

#############################################################################################################################
################################################### 최종 리포트 산출 #########################################################
#############################################################################################################################

def generate_financial_report(prediction, chain):
    report = chain.run({
        "industry": prediction["산업군"].iloc[0],  # Series에서 첫 번째 값을 가져옴
        "data_DEBT": prediction['2024_DEBT'].iloc[0] * 100,
        "data_ROA": prediction['2024_ROA'].iloc[0] * 100,
        "data_ATR": prediction['2024_ATR'].iloc[0] * 100,
        "data_AGR": prediction['2024_AGR'].iloc[0] * 100,
        "data_PPE": prediction['2024_PPE'].iloc[0] * 100,
        "last_DEBT": prediction['2023_DEBT'].iloc[0] * 100,
        "last_ROA": prediction['2023_ROA'].iloc[0] * 100,
        "last_ATR": prediction['2023_ATR'].iloc[0] * 100,
        "last_AGR": prediction['2023_AGR'].iloc[0] * 100,
        "last_PPE": prediction['2023_PPE'].iloc[0] * 100
    })
    return report


#############################################################################################################################
################################################ 리포트 마크다운 제거 #########################################################
#############################################################################################################################


def remove_special_characters(input_str):
    # Remove * and # characters
    result = input_str.replace('*', '').replace('#', '')
    return result
#############################################################################################################################
############################### 첫 데이터 프레임에서 우리가 쓸 데이터 프레임으로 변형  ###########################################
#############################################################################################################################


def make_DataFrame(df, col_name = ['DEBT', 'AGR', 'OCF', 'ROA', 'PPE', 'ART', 'ATR', 'OPM', 'GPM', 'ROE', 'NPM']):
    
    for col in col_name:
        if col == 'DEBT':
            fir_column = [col for col in df.columns if '부채총계' in col][0]
            sed_column = [col for col in df.columns if '자산총계' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1 = pd.DataFrame(ratio_series)
            df1.columns = [f'2023_{col}']
        elif col == 'AGR':
            fir_column = [col for col in df.columns if '전년도자산총계' in col][0]
            sed_column = [col for col in df.columns if '자산총계' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = (sed_series - fir_series) / fir_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'OCF':
            fir_column = [col for col in df.columns if '현금흐름' in col][0]
            sed_column = [col for col in df.columns if '자산총계' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'ROA':
            fir_column = [col for col in df.columns if '총당기순이익' in col][0]
            sed_column = [col for col in df.columns if '자산총계' in col][0]
            thr_column = [col for col in df.columns if '전년도자산총계' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            thr_series = df[thr_column]
            ratio_series = fir_series / ((sed_series + thr_series) / 2)
            df1[f'2023_{col}'] = ratio_series
        elif col == 'PPE':
            fir_column = [col for col in df.columns if '유형자산' in col][0]
            sed_column = [col for col in df.columns if '자산총계' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'ART':
            fir_column = [col for col in df.columns if '채권' in col][0]
            sed_column = [col for col in df.columns if '매출액' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'ATR':
            fir_column = [col for col in df.columns if '매출액' in col][0]
            sed_column = [col for col in df.columns if '자산총계' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'OPM':
            fir_column = [col for col in df.columns if '영업이익' in col][0]
            sed_column = [col for col in df.columns if '매출액' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'GPM':
            fir_column = [col for col in df.columns if '매출총이익' in col][0]
            sed_column = [col for col in df.columns if '매출액' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'ROE':
            fir_column = [col for col in df.columns if '총당기순이익' in col][0]
            sed_column = [col for col in df.columns if '자본총계' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'NPM':
            fir_column = [col for col in df.columns if '총당기순이익' in col][0]
            sed_column = [col for col in df.columns if '매출액' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        
    df1['Year'] = 2023
    df = df / 10000000000000
    df['Year'] = 2023
    
#####################################################################################################################################
######################################### 타깃(오각형) 마다 모델을 돌려 예측값 생성  ###################################################
#####################################################################################################################################

    timestep = 1 # 타임 스템 
    target_col = ['DEBT', 'AGR', 'ROA', 'PPE', 'ATR']

    #'DEBT', 'AGR', 'ROA', 'PPE', 'ATR'
    for col1 in target_col:
        if col1 == 'DEBT':
            target = f'2023_{col1}'
            x_val = df1.drop(target, axis = 1) 
            scaler_x_debt = joblib.load('scaler_x(DEBT) (1).pkl')
            x_val = scaler_x_debt.transform(x_val)
            x_val = np.array(x_val)
            x_val = temporalize(x_val, timestep)
            pred_debt = load_model('DEBT모델.keras').predict(x_val)
        elif col1 == 'AGR':
            target = f'2023_{col1}'
            x_val = df1.drop(target, axis = 1)
            scaler_x_agr = joblib.load('scaler_x(AGR) (1).pkl')
            x_val = scaler_x_agr.transform(x_val)
            x_val = np.array(x_val)
            x_val = temporalize(x_val, timestep)
            pred_agr = load_model('AGR모델.keras').predict(x_val)
        elif col1 == 'ROA':
            target = f'2023_{col1}'
            x_val = df1.drop(target, axis = 1)
            scaler_x_roa = joblib.load('scaler_x(ROA) (1).pkl')
            x_val = scaler_x_roa.transform(x_val)
            x_val = np.array(x_val)
            x_val = temporalize(x_val, timestep)
            pred_roa = load_model('ROA모델.keras').predict(x_val)
        elif col1 == 'PPE':
            target = f'2023_{col1}'
            x_val = df1.drop(target, axis = 1)
            scaler_x_ppe = joblib.load('scaler_x(PPE) (1).pkl')
            x_val = scaler_x_ppe.transform(x_val)
            x_val = np.array(x_val)
            x_val = temporalize(x_val, timestep)
            pred_ppe = load_model('PPE모델.keras').predict(x_val)
        elif col1 == 'ATR':
            target = f'2023_{col1}'
            x_val = df1.drop(target, axis = 1)
            scaler_x_atr = joblib.load('scaler_x(ATR) (1).pkl')
            x_val = scaler_x_atr.transform(x_val)
            x_val = np.array(x_val)
            x_val = temporalize(x_val, timestep)
            pred_atr = load_model('ATR모델.keras').predict(x_val)
            
            
    df1['2024_DEBT'] = pred_debt
    df1['2024_AGR'] = pred_agr
    df1['2024_ROA'] = pred_roa
    df1['2024_PPE'] = pred_ppe
    df1['2024_ATR'] = pred_atr
    
    df_copy = df1.copy()
    
#####################################################################################################################################
######################################### 전년도 비교 (4개) 모델을 돌려 예측값 생성  ###################################################
#####################################################################################################################################

    previous_target_col = ['매출액', '총당기순이익', '자산총계', '부채총계']
    df = df.drop([col for col in df.columns if '전년도' in col], axis = 1)

    for col2 in previous_target_col:
        if col2 == '매출액':
            x_val2 = df.drop([col for col in df.columns if f'{col2}' in col], axis = 1)
            scaler_x = joblib.load('scaler_x(총수익) (1).pkl')
            x_val2 = scaler_x.transform(x_val2)
            x_val2 = np.array(x_val2)
            x_val2 = temporalize(x_val2, timestep)
            pred_mae = load_model('총수익모델 (1).keras').predict(x_val2)
        elif col2 == '부채총계':
            x_val2 = df.drop([col for col in df.columns if f'{col2}' in col], axis = 1)
            scaler_x = joblib.load('scaler_x(부채총계) (1).pkl')
            x_val2 = scaler_x.transform(x_val2)
            x_val2 = np.array(x_val2)
            x_val2 = temporalize(x_val2, timestep)
            pred_bu = load_model('부채총계모델 (1).keras').predict(x_val2)
        elif col2 == '총당기순이익':
            x_val2 = df.drop([col for col in df.columns if f'{col2}' in col], axis = 1)
            scaler_x = joblib.load('scaler_x(당기순이익) (1).pkl')
            x_val2 = scaler_x.transform(x_val2)
            x_val2 = np.array(x_val2)
            x_val2 = temporalize(x_val2, timestep)
            pred_dang = load_model('당기순이익모델 (1).keras').predict(x_val2)
        elif col2 == '자산총계':
            x_val2 = df.drop([col for col in df.columns if f'{col2}' in col], axis = 1)            
            scaler_x = joblib.load('scaler_x(자산총계) (1).pkl')
            x_val2 = scaler_x.transform(x_val2)
            x_val2 = np.array(x_val2)
            x_val2 = temporalize(x_val2, timestep)
            pred_jasan = load_model('자산총계모델 (1).keras').predict(x_val2)

    df_copy['2023_당기순이익'] = df[[col for col in df.columns if '당기순이익' in col]]
    df_copy['2023_자산총계'] = df[[col for col in df.columns if '자산총계' in col]]
    df_copy['2023_부채총계'] = df[[col for col in df.columns if '부채총계' in col]]
    df_copy['2023_매출액'] = df[[col for col in df.columns if '매출액' in col]]
    
    df_copy['2024_당기순이익예측'] = pred_dang
    df_copy['2024_자산총계예측'] = pred_jasan
    df_copy['2024_부채총계예측'] = pred_bu
    df_copy['2024_매출액예측'] = pred_mae
    
####################################################################################################################################
############################################### 생성형 AI 모델 및 LangChain 함수 설정  ###############################################
####################################################################################################################################

    # LangChain 설정
    llm = ChatOpenAI(model="gpt-4o", api_key=os.getenv('OPENAI_API_KEY'))

    template = """
    Describe and analyze the financial data based on the actual data and forecast data for 2023. Provide a detailed financial analysis report including specific solutions tailored to the industry. The report should include the following:

    - Forecast Debt Ratio (DEBT): {data_DEBT: .2f}%
    - Forecast Return on Assets (ROA): {data_ROA: .2f}%
    - Forecast Asset Turnover Ratio (ATR): {data_ATR: .2f}%
    - Forecast Asset Growth Rate (AGR): {data_AGR: .2f}%
    - Forecast Property, Plant, and Equipment Ratio (PPE): {data_PPE: .2f}%
    - 2023 Debt Ratio: {last_DEBT: .2f}%
    - 2023 Return on Assets: {last_ROA: .2f}%
    - 2023 Asset Turnover Ratio: {last_ATR: .2f}%
    - 2023 Asset Growth Rate: {last_AGR: .2f}%
    - 2023 Property, Plant, and Equipment Ratio: {last_PPE: .2f}%
    - Industry: {industry}

    Provide a detailed explanation for each metric, including:
    1. Evaluation and industry-specific solutions
    2. Impact on overall financial condition
    3. Final conclusion

    Format the explanation for each metric as follows:

    - Debt Ratio (DEBT):
      1. Impact:
         - A high debt ratio can lead to increased financial risk, potentially affecting the company’s credit rating and interest expenses.
         - Conversely, a lower debt ratio might indicate under-leverage, which could impact growth opportunities.
      2. Evaluation and Industry-Specific Solutions: 
         - The debt ratio of {data_DEBT: .2f}% compared to the previous year's {last_DEBT: .2f}% indicates a notable shift in the company's financial leverage. This suggests a strategic move towards either increasing or reducing debt to balance financial stability and growth potential. In the {industry} industry, companies often face volatile market conditions, and maintaining an optimal debt ratio is crucial. It is recommended to optimize the capital structure by balancing between equity and debt, ensuring the company leverages debt effectively for growth while minimizing financial risks. Strategies such as refinancing high-interest debts or renegotiating debt terms can also help in managing the debt load efficiently.
      3. Conclusion:
         The current debt ratio suggests that the company should focus on strategic debt management. This includes actions such as refinancing high-interest debts and negotiating better terms to reduce financial risks. Maintaining an optimal balance between debt and equity is crucial to ensure sustainable growth. The company needs to continuously monitor its debt levels and adjust its financial strategies to respond to changing market conditions and opportunities for growth, ensuring that the debt load is manageable and supports long-term financial stability.

    - Return on Assets (ROA):
      1. Impact: 
         A higher ROA indicates efficient use of assets to generate profit, improving overall profitability and attractiveness to investors.
         A lower ROA might signal inefficiencies and require strategic adjustments.
      2. Evaluation and Industry-Specific Solutions: 
         With a ROA of {data_ROA: .2f}% compared to the previous year's {last_ROA: .2f}%, it is evident that the company has either enhanced or deteriorated its efficiency in utilizing its assets to generate profit. In the {industry} industry, where asset-heavy operations are common, maintaining a high ROA is indicative of robust operational performance. To further improve ROA, companies should enhance the utilization of existing assets through better management practices and operational improvements. Additionally, investing in high-yield assets and divesting underperforming assets can help optimize returns.
      3. Conclusion: 
         The ROA suggests that the company should focus on enhancing operational efficiencies and optimizing asset management. This includes investing in high-yield assets and divesting underperforming ones to ensure maximum return on asset utilization. By implementing better management practices and leveraging technology to improve operational efficiency, the company can enhance its profitability and competitive position in the industry.

    - Asset Turnover Ratio (ATR):
      1. Impact:
         A higher ATR means better efficiency in using assets to generate sales, contributing to overall growth.
         A lower ATR suggests potential inefficiencies in asset utilization.
      2. Evaluation and Industry-Specific Solutions: 
         The ATR of {data_ATR: .2f}% compared to the previous year's {last_ATR: .2f}% reflects significant changes in how efficiently the company utilizes its assets to generate sales. In the {industry} industry, a high ATR is often associated with strong operational efficiency and effective asset management. Companies should focus on streamlining operations to improve the turnover of assets and reducing idle assets. Implementing strategies to boost sales without significant increases in assets can also enhance the ATR.
      3. Conclusion: 
         The ATR analysis shows that the company needs to improve its asset utilization to generate higher sales. Streamlining operations and reducing idle assets are crucial steps. By implementing sales enhancement strategies without substantial asset growth, the company can improve its ATR, leading to better overall financial health and competitiveness in the industry. Continuous monitoring and adjustment of asset management strategies will help in maintaining an optimal ATR.

    - Asset Growth Rate (AGR):
      1. Impact:
         A high AGR indicates strong growth potential and expansion capabilities, appealing to investors and stakeholders.
         A low or negative AGR might suggest stagnation or contraction, requiring strategic interventions.
      2. Evaluation and Industry-Specific Solutions: 
         The AGR of {data_AGR: .2f}% compared to the previous year's {last_AGR: .2f}% indicates the pace at which the company is expanding its asset base. For companies in the {industry} industry, sustaining a strong AGR is essential for staying competitive and pursuing growth opportunities. To achieve this, companies should pursue strategic acquisitions, partnerships, and new market entries to drive asset growth. Additionally, focusing on optimizing existing assets can drive organic growth.
      3. Conclusion: 
         The AGR suggests that the company should focus on strategic acquisitions and partnerships to boost asset growth. Exploring new market entries and optimizing the use of existing assets will also contribute to organic growth. Maintaining a strong AGR is vital for the company's competitiveness and ability to seize new opportunities. Regular review of growth strategies and market conditions will help the company adapt and sustain its growth trajectory.

    - Property, Plant, and Equipment Ratio (PPE):
      1. Impact:
         A higher PPE ratio suggests significant investments in fixed assets, which could be beneficial for production capacity but also increases fixed costs.
         A lower PPE ratio indicates fewer investments in fixed assets, which might limit production capabilities.
      2. Evaluation and Industry-Specific Solutions: 
         The PPE ratio of {data_PPE: .2f}% compared to the previous year's {last_PPE: .2f}% reflects how the company allocates its investments in fixed assets. In the {industry} industry, where significant investments in production capacity are common, maintaining an optimal PPE ratio is crucial. Companies should manage capital expenditures on PPE to balance between growth and cost management. Improving the utilization rates of existing PPE to enhance productivity and return on investment is also vital.
      3. Conclusion:
         The PPE ratio analysis indicates that the company needs to balance its investments in fixed assets with cost management. Enhancing the utilization rates of existing PPE can improve productivity and return on investment. By carefully planning capital expenditures and focusing on efficient use of existing assets, the company can achieve a balance that supports growth while managing costs effectively. Continuous assessment of PPE investments and their impact on the company's financial performance will help maintain an optimal PPE ratio.

    Note: Please output the final report in Korean.
    """


    prompt = PromptTemplate(
        input_variables=["industry", "data_DEBT", "data_ROA", "data_ATR", "data_AGR", "data_PPE"],
        template=template
    )

    chain = LLMChain(llm=llm, prompt=prompt)

    df1['산업군'] = str(input())
    
    # 최종 보고서 생성
    final_report = generate_financial_report(df1, chain)

    
    cleaned_string = remove_special_characters(final_report)
    print(cleaned_string)
    df_copy['예측 기업 평가 보고서'] = cleaned_string
    
    return df_copy

In [30]:
df_final = make_DataFrame(df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 515ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 593ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 518ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 619ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 629ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 402ms/step


 자동차


 2023년 재무 분석 보고서

 부채비율 (DEBT):
1. 영향:
   - 높은 부채비율은 재무 리스크를 증가시켜 회사의 신용 등급 및 이자 비용에 영향을 미칠 수 있습니다.
   - 반대로, 낮은 부채비율은 레버리지 부족을 나타내며 성장 기회를 제한할 수 있습니다.

2. 평가 및 산업 특화 솔루션:
   - 2023년 부채비율 44.23%는 전년도의 20.00%와 비교하여 상당한 변화가 있음을 나타냅니다. 이는 회사가 재정적 안정성과 성장 잠재력을 균형 있게 유지하기 위해 부채를 늘리거나 줄이는 전략적 움직임을 보였음을 시사합니다. 자동차 산업에서는 변동성이 큰 시장 상황에 대응하기 위해 최적의 부채비율을 유지하는 것이 중요합니다. 자본 구조 최적화를 통해 부채와 자본의 균형을 맞추고, 고이율 부채의 재융자나 부채 조건 재협상을 통해 부채 부담을 효율적으로 관리하는 것이 필요합니다.

3. 결론:
   현재의 부채비율은 회사가 전략적인 부채 관리를 집중해야 함을 시사합니다. 고이율 부채의 재융자 및 더 나은 조건 재협상과 같은 행동을 통해 재무 리스크를 줄이는 것이 중요합니다. 부채와 자본의 균형을 유지하여 지속 가능한 성장을 보장하고, 시장 조건과 성장 기회를 고려하여 재무 전략을 지속적으로 조정하는 것이 필요합니다.

 자산 수익률 (ROA):
1. 영향:
   - 높은 ROA는 자산을 효율적으로 사용해 이익을 창출하는 것을 나타내며, 전반적인 수익성과 투자자에게 매력적임을 보여줍니다.
   - 낮은 ROA는 비효율성을 나타내며 전략적 조정을 필요로 할 수 있습니다.

2. 평가 및 산업 특화 솔루션:
   - 2023년 ROA는 7.82%로, 전년도의 6.32%와 비교하여 회사가 자산을 이용해 이익을 창출하는 효율성이 개선되었음을 나타냅니다. 자동차 산업에서는 자산 집약적인 운영이 일반적이므로 높은 ROA는 견고한 운영 성과를 나타냅니다. ROA를 더욱 개선하기 위해서는 기존 자산의 활용도를 높이고, 운영 효율성을 향상시키며, 고수익 자산에 투자하

In [26]:
df_final.columns

Index(['2023_DEBT', '2023_AGR', '2023_OCF', '2023_ROA', '2023_PPE', '2023_ART',
       '2023_ATR', '2023_OPM', '2023_GPM', '2023_ROE', '2023_NPM', 'Year',
       '2024_DEBT', '2024_AGR', '2024_ROA', '2024_PPE', '2024_ATR',
       '2023_당기순이익', '2023_자산총계', '2023_부채총계', '2023_매출액', '2024_당기순이익예측',
       '2024_자산총계예측', '2024_부채총계예측', '2024_매출액예측', '예측 기업 평가 보고서'],
      dtype='object')